## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Alta,AR,-38.8759,-62.0736,64.20,34,1,16.87,clear sky
1,1,Hermanus,ZA,-34.4187,19.2345,53.80,61,0,5.82,clear sky
2,2,Cidreira,BR,-30.1811,-50.2056,79.61,73,10,8.84,clear sky
3,3,Lebu,CL,-37.6167,-73.6500,54.72,61,39,10.22,scattered clouds
4,4,Fairbanks,US,64.8378,-147.7164,48.20,43,0,4.61,clear sky


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Cidreira,BR,-30.1811,-50.2056,79.61,73,10,8.84,clear sky
10,10,Teahupoo,PF,-17.8333,-149.2667,76.96,77,100,25.50,light rain
12,12,Rikitea,PF,-23.1203,-134.9692,77.16,72,7,24.67,clear sky
16,16,Bolobo,CD,-2.1667,16.2333,75.56,80,94,1.50,overcast clouds
26,26,Isangel,VU,-19.5500,169.2667,76.57,78,100,13.35,overcast clouds
28,28,Sao Joao Da Barra,BR,-21.6403,-41.0511,84.29,81,18,11.61,few clouds
31,31,Lincoln,US,40.8000,-96.6670,80.51,42,0,14.97,clear sky
34,34,Puerto Ayora,EC,-0.7393,-90.3518,78.76,77,30,8.28,scattered clouds
35,35,Daryapur,IN,20.9333,77.3333,95.45,13,28,7.34,scattered clouds
37,37,Bubaque,GW,11.2833,-15.8333,78.21,78,5,14.65,clear sky


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_nonull_df = preferred_cities_df.dropna()
preferred_cities_nonull_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_nonull_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Cidreira,BR,79.61,clear sky,-30.1811,-50.2056,
10,Teahupoo,PF,76.96,light rain,-17.8333,-149.2667,
12,Rikitea,PF,77.16,clear sky,-23.1203,-134.9692,
16,Bolobo,CD,75.56,overcast clouds,-2.1667,16.2333,
26,Isangel,VU,76.57,overcast clouds,-19.5500,169.2667,
28,Sao Joao Da Barra,BR,84.29,few clouds,-21.6403,-41.0511,
31,Lincoln,US,80.51,clear sky,40.8000,-96.6670,
34,Puerto Ayora,EC,78.76,scattered clouds,-0.7393,-90.3518,
35,Daryapur,IN,95.45,scattered clouds,20.9333,77.3333,
37,Bubaque,GW,78.21,clear sky,11.2833,-15.8333,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
         print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
